---
## Section 3: Prompt Engineering (~25 min)
---

### 3.1 Prompt Engineering Fundamentals

The prompt is the **most critical component** of a RAG system. It controls:
- Whether the LLM stays grounded in context or hallucinates
- The format and style of answers
- How the model handles unanswerable questions

**Anatomy of a RAG prompt:**
```
ROLE:         "You are a helpful AI course instructor..."
INSTRUCTIONS: "Use ONLY the provided context..."
CONTEXT:      "{context}"  ← filled by retrieval
QUESTION:     "{question}" ← filled by user
```

We'll design and compare 4 strategies:

| Strategy | Key Idea | Tradeoff |
|----------|----------|----------|
| **Restrictive** | Context-only, refuse if not found | Safe but may refuse valid questions |
| **Permissive** | Context-primary, supplement with knowledge | More helpful but less grounded |
| **Few-shot** | Include example Q&A pairs | Consistent style but longer prompt |
| **Structured** | Force Answer/Details/Source format | Predictable format but rigid |

### 3.2 Designing Prompt Strategies

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Prompt 1: Restrictive
# Strictly grounded in context only. Refuses if answer not found.
restrictive_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a chemical safety specialist that answers "
     "questions about Covestro Safety Data Sheets (SDS) and material safety information.\n\n"
     "Context:\n{context}"),
    ("human", "{question}"),
])

print("Prompt 1 (Restrictive): Strict context-only grounding")

Prompt 1 (Restrictive): Strict context-only grounding


### 3.3 Comparing Prompts

In [ ]:
def create_rag_with_prompt(prompt_template: ChatPromptTemplate) -> StateGraph:
    """Build a basic RAG graph with a specific prompt."""
    def _retrieve(state: State) -> dict:
        docs = vector_store.similarity_search(state["question"], k=3)
        context = "\n\n".join(doc.page_content for doc in docs)
        return {"context": context}
    
    def _generate(state: State) -> dict:
        messages = prompt_template.format_messages(
            context=state["context"],
            question=state["question"],
        )
        response = llm.invoke(messages)
        return {"answer": response.content}
    
    graph = StateGraph(State)
    graph.add_node("retrieve", _retrieve)
    graph.add_node("generate", _generate)
    graph.add_edge(START, "retrieve")
    graph.add_edge("retrieve", "generate")
    graph.add_edge("generate", END)
    return graph.compile()

prompts = {
    "Restrictive": restrictive_prompt,
    "Permissive": permissive_prompt,
    "Few-shot": few_shot_prompt,
    "Structured": structured_prompt,
}

graphs = {name: create_rag_with_prompt(p) for name, p in prompts.items()}
print(f"Built {len(graphs)} graph variants")

Built 4 graph variants


In [ ]:
test_question = "What PPE is required for DESMOPHEN XP 2680?"
print(f"Question: {test_question}\n")

for name, graph in graphs.items():
    result = graph.invoke({"question": test_question})
    print(f"{'='*60}")
    print(f"Strategy: {name}")
    print(f"{'='*60}")
    print(result["answer"])
    print()

Question: What PPE is required for DESMOPHEN XP 2680?

Strategy: Restrictive
I don't have enough information in the provided context to answer this question.

Strategy: Permissive
Based on the provided context, I do not have specific information about the Personal Protective Equipment (PPE) requirements for DESMOPHEN XP 2680.

However, as a general guideline, polyol systems like DESMOPHEN XP 2680 are typically handled in well-ventilated areas and may require PPE to minimize exposure to skin contact and inhalation of dust or fumes. 

Based on general knowledge: 

* Gloves (e.g., nitrile or butyl rubber) should be worn to prevent skin contact.
* Safety glasses or goggles should be used to protect eyes from splashes or spills.
* A face mask or respirator may be necessary in areas with high dust concentrations or during application.

Please consult the Material Safety Data Sheet (MSDS) or Safety Data Sheet (SDS) for DESMOPHEN XP 2680, which is not provided in the context. The SDS will prov

In [ ]:
oos_question = "How do I deploy a Kubernetes cluster on AWS?"
print(f"Out-of-scope question: {oos_question}\n")

for name, graph in graphs.items():
    result = graph.invoke({"question": oos_question})
    print(f"{'='*60}")
    print(f"Strategy: {name}")
    print(f"{'='*60}")
    print(result["answer"][:300])
    print()

### 3.4 Choosing the Best Prompt

**Selection criteria:**

| Criteria | Restrictive | Permissive | Few-shot | Structured |
|----------|:-----------:|:----------:|:--------:|:----------:|
| Grounding (no hallucination) | Best | Weakest | Good | Good |
| Informativeness | Limited | Best | Good | Good |
| Format consistency | Low | Low | Medium | Best |
| Refusal handling | Best | Weakest | Good | Good |

For production RAG, **Restrictive** is often the safest default. **Structured** is great when you need predictable formatting. We'll carry the **Restrictive** prompt forward as our default.

In [ ]:
selected_prompt = restrictive_prompt
print("Selected prompt: Restrictive (safest for production)")

Selected prompt: Restrictive (safest for production)


### Module checkpoint

All 4 prompt templates and `get_prompt(style)` are in `rag/prompts.py`.